In [1]:
import matplotlib.pyplot as plt
from numpy import genfromtxt
import seaborn as sns
import os
import numpy as np
from matplotlib import cm
from sklearn.cluster import MeanShift

In [2]:
experiments_mcmc_nj = [
                    "simple_nj_c5_d2",
                    "simple_nj_c5_d3",
                    "simple_nj_c5_d4",
                    "simple_nj_c5_d5",
                    "simple_nj_c5_d6",
                    "simple_nj_c5_d10"
                    ]
experiments_mcmc_geo = [
                    "simple_geodesics_c5_d2",
                    "simple_geodesics_c5_d3",
                    "simple_geodesics_c5_d4",
                    "simple_geodesics_c5_d5",
                    "simple_geodesics_c5_d6",
                    "simple_geodesics_c5_d10"
                    ]
dims = (2, 3, 4, 5, 6, 10)
burnin = 600
experiments_vi = []
dir = "../../data/T17"


In [9]:
fig, ax = plt.subplots(1, 1)
ax.set_yscale('log')
handles = 3*[None]

# plot MrBayes as line
fn = "dna.nex.run1.p"
path = os.path.join(dir, "mrbayes", fn)
data = genfromtxt(path, skip_header=2)
posterior = data[burnin:, 1] + data[burnin:, 2]
map = MeanShift().fit(posterior.reshape(-1, 1))
# plt.plot([min(dims), max(dims)], [-map.cluster_centers_[0], -map.cluster_centers_[0]], color='k', label='MrBayes')
mb_base = map.cluster_centers_[0]

for ext in ("nj", "geo"):
    if ext == "nj": 
        experiments_mcmc = experiments_mcmc_nj
    else:
        experiments_mcmc = experiments_mcmc_geo
    paths = [os.path.join(dir, 'mcmc', e, 'mcmc.trees') for e in experiments_mcmc]
    for e in experiments_vi:
        paths.append("%s/vi/%s/vi.trees" % (dir, e))
    n_lines = len(paths)

    for count, path in enumerate(paths):
        lnL = []
        lnPr = []
        iter = []
        readLine = False
        for i, line in enumerate(open(path)):
            if line.startswith('tree STATE_'):
                _, _, after = line.partition('STATE_')
                iter_, _, _ = after.partition(' ')
                iter.append(int(iter_))

                _, _, after = line.partition('&lnL=')
                lnL_, _, _ = after.partition(', ')
                lnL.append(float(lnL_))

                _, _, after = line.partition('&lnPr=')
                lnPr_, _, _ = after.partition(']')
                lnPr.append(float(lnPr_))
        
        posterior = np.array(lnPr) + np.array(lnL)
        data = posterior[burnin:].reshape(-1, 1)
        map = MeanShift().fit(data)
        if ext == "nj": 
            plt.plot(dims[count], (map.cluster_centers_[0]-mb_base)/mb_base, 'o', color='b')
            if count == 0:
                plt.plot(dims[count], (map.cluster_centers_[0]-mb_base)/mb_base, 'o', color='b', label="NJ")
        else:
            plt.plot(dims[count], (map.cluster_centers_[0]-mb_base)/mb_base, 'o', color='r')
            if count == 0:
                plt.plot(dims[count], (map.cluster_centers_[0]-mb_base)/mb_base, 'o', color='r', label="Geodesics")

ax.set_xlabel("Dimension")
ax.set_ylabel("$\mathregular{(MAP_{mb} - MAP_i)/MAP_{mb}}$")
plt.legend()
plt.show()
